In [22]:
import pandas as pd
import re
import os

In [23]:
# This function creates the path to the main Excel file of this project
# Uses os library methods to ensure the file path works in both local and GitHub CI/CD environments
def create_file_path(file_relative_path):
    # Store the xlsx file dir into a var
    dir = '../data'
    
    # Iterate under dir data to have all files
    for filename in os.listdir(dir):
        # Condition to confirm which file is == year_make_model_df.xlsx
        if filename.endswith(file_relative_path):
            # Join the dir + filename to create the path to year_make_model_df.xlsx file  
            xlsx_file_fullpath = os.path.join(dir, filename)
            # Print the result
            print(xlsx_file_fullpath)
    return xlsx_file_fullpath

In [24]:
# Read excel 
df = pd.read_excel(create_file_path('year_make_model_df.xlsx'))

../data\year_make_model_df.xlsx


In [25]:
print(set(df['Make']))

{'Chrysler', 'Oldsmobile', 'Plymouth', 'Lincoln', 'Mercury', 'GMC', 'Mini', 'Hummer', 'Cadillac', 'Mazda', 'Jeep', 'Scion', 'Rolls-Royce', 'Infiniti', 'Lexus', 'Pontiac', 'Nissan', 'Volkswagen', 'Jaguar', 'Buick', 'Saturn', 'Ford', 'Mitsubishi', 'Chevrolet', 'Land Rover', 'Fiat', 'Toyota', 'Subaru', 'Dodge', 'BMW', 'Audi', 'Acura', 'Honda'}


In [26]:
# Converting all values in column Model to string, as some of models are numbers, for instance BMWs 318, 328, 525
df = df.astype({'Model':'string'})

In [27]:
# Confirm if the Dtype from the column Model changed to string.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3463 entries, 0 to 3462
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Year            3463 non-null   int64 
 1   Make            3463 non-null   object
 2   Model           3463 non-null   string
 3   Security        3156 non-null   object
 4   ParameterReset  938 non-null    object
dtypes: int64(1), object(3), string(1)
memory usage: 135.4+ KB


In [28]:
# Function to create list with unique items from given column
def data_from_column(df, column):
    # List to append the items
    data_column_list =[]

    # Iterate over given column
    for data in df[column]:
        # Append the items from the given column
        data_column_list.append(data)

    # Return a set with unique items from given column
    return set(data_column_list)

In [29]:
# Call the function to create a set with unique values from column make
print(data_from_column(df, "Make"))

{'Chrysler', 'Oldsmobile', 'Plymouth', 'Lincoln', 'Mercury', 'GMC', 'Mini', 'Hummer', 'Cadillac', 'Mazda', 'Jeep', 'Scion', 'Rolls-Royce', 'Infiniti', 'Lexus', 'Pontiac', 'Nissan', 'Volkswagen', 'Jaguar', 'Buick', 'Saturn', 'Ford', 'Mitsubishi', 'Chevrolet', 'Land Rover', 'Fiat', 'Toyota', 'Subaru', 'Dodge', 'BMW', 'Audi', 'Acura', 'Honda'}


In [30]:
# Check which models the df has and print only the unique values
models = df['Model'].unique()

In [31]:
# Create a new dataframe containing only Chevrolet: 'Chevrolet', 'Pontiac', 'Cadillac', 'Buick', 'Oldsmobile', 'GMC', 'Saturn', 'Hummer'
chevrolet_list = ['Chevrolet', 'Pontiac', 'Cadillac', 'Buick', 'Oldsmobile', 'GMC', 'Saturn', 'Hummer']

In [32]:
# Create a dataframe containing only chevrolet brands 
df_chevrolet = df[df['Make'].isin(chevrolet_list)]

In [33]:
# Drop the column 'ParameterReset' as it is not applicable to Chevrolet
df_chevrolet_dropped_column = df_chevrolet.drop(columns=['ParameterReset'])

In [34]:
# Rename column 'Security'
df_chevrolet_dropped_column.rename(columns={'Security': 'Security Type'})

,Year,Make,Model,Security Type
0,1986,Chevrolet,Corvette,VATS
1,1987,Chevrolet,Corvette,VATS
2,1988,Chevrolet,Corvette,VATS
3,1988,Pontiac,Firebird,GTA\nonly
4,1989,Cadillac,Allante,PK1
...,...,...,...,...
3284,2013,GMC,Savana,PK3+
3285,2013,GMC,Sierra,PK3+
3286,2013,GMC,Yukon XL,PK3+
3287,2013,GMC,Yukon/Denali,PK3+


## Add hardware clonable info
A new column named 'Is this hardware clonable?' will be included in the Chevrolet dataframe to show which hardwares are clonable based on the part number. As the hardware part numbers are not available and their values will need to be added manually to the main df, these are the next steps to add the hardware part numbers:
1. Export the main df (df_chevrolet_dropped_column) to excel file.
2. Add the hardware part numbers manually from the GM compatibility chart.
3. Fill out the column 'Is this hardware clonable?' rows comparing the 'df_chevrolet_dropped_column' with the 'List of Cloneable Modules'. 

In [13]:
# Download the dateframe 'chevrolet_list_filtered' in xlsx format and name the file 'chevrolet_hardware_list
# df_chevrolet_dropped_column.to_excel('chevrolet_hardware_list.xlsx', index=False)

In [35]:
# Load the xlsx file containing the dataframe after adding the hardware part numbers
df_chevrolet = pd.read_excel(create_file_path('chevrolet_hardware_list.xlsx'))

../data\chevrolet_hardware_list.xlsx


In [36]:
# Load the xlsx file containing the clonable hardware part numbers   
list_clonable_modules = pd.read_excel(create_file_path('List of Clonable Modules.xlsx'))

../data\List of Clonable Modules.xlsx


In [37]:
# Show all columns of the dataframe
list_clonable_modules.columns

Index(['Cloning Tools', 'ECM Hardware', 'cloning_status', 'ECU Brand', 'Year',
       'Year/Model/Engine Size/HP', 'Manufacturer', 'Vehicle Type', 'Notes'],
      dtype='object')

In [17]:
# Drop most columns and leave only 2 columns: 'ECM Hardware' and 'cloning_status'
list_clonable_modules_updated = list_clonable_modules.drop(columns=['Cloning Tools', 'ECU Brand', 'Year', 'Year/Model/Engine Size/HP', 'Manufacturer', 'Vehicle Type', 'Notes'])

In [18]:
# Print the first rows on the dataframe updated
list_clonable_modules_updated.head()

,ECM Hardware,cloning_status
0,E83 (GM),Currently possible
1,E78 (GM),Currently possible
2,E87 (GM),Currently possible
3,E39 (GM),Currently possible
4,E39A (GM),Currently possible


In [19]:
# Filter the hardware pn# that can be clonable
list_clonable_modules_filtered = list_clonable_modules_updated[list_clonable_modules_updated['cloning_status'] == 'Currently possible']

In [20]:
# Create a list to store the hardwares prefixes got from the clonable list
clonable_modules_list = []

# Iterate in the column 'ECM Hardware' to get their prefixes
for part_number in list_clonable_modules_filtered['ECM Hardware']:
    ecm_hardware_prefix = part_number.split(' ')[0]
    clonable_modules_list.append(ecm_hardware_prefix)

In [21]:
# Print the last rows on the dataframe containing the hardware part number values
df_chevrolet.tail()

,Year,Make,Model,Security,hardware_part_number
657,2013,GMC,Savana,PK3+,E38-AK8
658,2013,GMC,Sierra,PK3+,E86-A24
659,2013,GMC,Yukon XL,PK3+,Information not available
660,2013,GMC,Yukon Denali,PK3+,E38-AK8
661,2013,GMC,Terrain,PK3+\nSmart Key,E39A-H98S


In [22]:
# Create a list to store hardware prefixes
hardware_prefix_list = []

# Iterate into the main dataframe to get the prefixes 
for hardware_pn in df_chevrolet['hardware_part_number']:
    hardware_prefix = hardware_pn.split('-')[0] # [0] indicates the first part of the list where the string needed is located, eg: 'E39', 'E38', etc.
    hardware_prefix_list.append(hardware_prefix)
    
# Add a new column contaning the prefixes
df_chevrolet['hardware_prefix'] = hardware_prefix_list

In [23]:
# Create a list to store the the string "Yes" or "No"
hardware_clonable_list = []
# Variable containing the string "Yes" to be used when a given part number is in the clonable_modules_list
yes_clonable = 'Yes'
# Variable containing the string "No" to be used when a given part number IS NOT in the clonable_modules_list
not_clonable = 'No'
# Loop in the 'hardware_prefix' column to check whether or not the part numbers are clonable 
for hardware_pn in df_chevrolet['hardware_prefix']:
    if hardware_pn in clonable_modules_list:
        hardware_clonable_list.append(yes_clonable)
    else:
        hardware_clonable_list.append(not_clonable)

# Create a new column after confirming the part numbers are clonable or not 
df_chevrolet['Is this hardware clonable?'] = hardware_clonable_list

In [24]:
# Drop column 'hardware_prefix' as it was used only to confirm the clonable hardwares
df_chevrolet_dropped_hardwarepre = df_chevrolet.drop(columns='hardware_prefix')

In [25]:
# Call the function to check the string from column Make
data_from_column(df_chevrolet_dropped_hardwarepre, "Make")

{'Buick',
 'Cadillac',
 'Chevrolet',
 'GMC',
 'Hummer',
 'Oldsmobile',
 'Pontiac',
 'Saturn'}

In [26]:
# Call the function to check the string from column Model
print(data_from_column(df_chevrolet_dropped_hardwarepre, "Model"))

{'Caprice/SS', 'C/K Truck', 'Impala', 'Aurora', 'Cobalt', 'Escalade ESV', 'Vue', 'Eldorado', 'Terrain', 'Volt', 'Bonneville', 'H3', 'Montana', 'Savana', 'Monte Carlo', 'Achieva', 'Bravada', 'Envoy', 'L Series', 'Aztek', 'Escalade', 'G5', 'G3', 'Sierra', 'Roadmaster', 'Sonic', 'H2', 'Encore', 'Spark', 'Cutlass Supreme', 'Terraza', 'Rendezvous', 'Brougham', 'G8', 'Torrent', 'SRX', 'Aura', 'ION', 'Intrigue', 'LaCrosse', 'Tahoe', 'Malibu', 'Sonoma', 'Equinox', 'Suburban', 'Classic', 'H2 SUT', 'Envoy XUV', 'Camaro', 'CTS', 'Venture', 'Colorado', 'Rainier', 'Sunfire', 'DTS', 'Reatta', 'ATS', 'Yukon/Denali', 'Verano', 'Corvette', 'Escalade EXT', 'Montana SV6', 'Express', 'Deville', 'Caprice (Police)', 'G6', 'Yukon Denali', 'Skylark', 'Park Avenue', 'Vibe', 'Solstice', 'Sky', 'Acadia', 'Uplander', 'Lumina', 'Allante', 'Firebird', 'Eighty-Eight', 'Astra', 'LeSabre', 'Ninety-Eight', 'Riviera', 'GTO', 'Trailblazer', 'Enclave', 'Relay', 'Regal', 'Yukon XL', 'Captiva', 'XLR', 'HHR', 'Safari', 'Cava

In [27]:
# Call the function to check the string from column Security
data_from_column(df_chevrolet_dropped_hardwarepre, "Security")

{'EZ',
 'GTA\nonly',
 'Holden',
 'KA',
 'Opel',
 'Opt',
 'PK1',
 'PK2',
 'PK3',
 'PK3+',
 'PK3+\nSmart Key',
 'PK3+/KA',
 'PK3/KA+',
 'PL1',
 'PL1 (GT\nonly*)',
 'PL1(Z24\nonly*)',
 'PL2',
 'PL3',
 'PL3/PK3+',
 'Toyota\nImmobilizer',
 'VATS',
 nan}

## Changes needed:
- Remove chars: \n  +/  /  **+**
- Remove all strings and leave only the GM security code such as PL1, PK3, PK2 when they are present.
- Remove PL3 from the ones which has PK3 

In [28]:
# Create a df containing only rows with PL, PK2 and PK3 strings
# r'\1' - using the replace method, it will replace by the 1st captured group
# ^ - beginning of the string
# [A-Z]{2} - 2 digits caps letters
# [0-9] - number 
# .* - all chars 0 or more times
# re.DOTALL - consider all chars including break lines
df_chevrolet_dropped_hardwarepre['Security'] = df_chevrolet_dropped_hardwarepre['Security'].str.replace(r'(^[A-Z]{2}[0-9]).*', r'\1', regex=True, flags=re.DOTALL) 

In [29]:
# Replace break line (\n) for whitespace and empty rows
df_chevrolet_dropped_hardwarepre['Security'] = df_chevrolet_dropped_hardwarepre['Security'].str.replace('\n', ' ', regex=True).fillna('Information not available')

In [30]:
# Check if all changes were done
data_from_column(df_chevrolet_dropped_hardwarepre, 'Security')

{'EZ',
 'GTA only',
 'Holden',
 'Information not available',
 'KA',
 'Opel',
 'Opt',
 'PK1',
 'PK2',
 'PK3',
 'PL1',
 'PL2',
 'PL3',
 'Toyota Immobilizer',
 'VATS'}

## CREATING SEPARATE DATAFRAMES FOR EACH MAKE:
 
As most makes have different security system and information-related, it is needed to create separate Dataframes for all/most of them. 

For instance, Ford vehicles requires parameter reset and there is a column showing whether or not this procedure is required. However, Chevrolet brands requires a different procedure and some hardwares can be cloned. That is, Ford and Chevrolet vehicles do not match the security information-related. The solution for this will be to create separate Dataframes for all makes.   

In [31]:
# Show all makes under Chevrolet and use this list to guide when creating separate Dataframes
df_chevrolet_dropped_hardwarepre['Make'].unique()

array(['Chevrolet', 'Pontiac', 'Cadillac', 'Buick', 'Oldsmobile', 'GMC',
       'Saturn', 'Hummer'], dtype=object)

In [32]:
# Print the columns
df_chevrolet_dropped_hardwarepre.columns

Index(['Year', 'Make', 'Model', 'Security', 'hardware_part_number',
       'Is this hardware clonable?'],
      dtype='object')

In [33]:
# Rename the column 'hardware_part_number' with 'Hardware Part Number'
df_chevrolet_renamed_column = df_chevrolet_dropped_hardwarepre.rename(columns={'hardware_part_number':'Hardware Part Number'})

In [34]:
# Create a dataframe only with Make: Chevrolet
df_chevrolet_final_version = df_chevrolet_renamed_column[df_chevrolet_renamed_column['Make'] == 'Chevrolet']

In [35]:
# Check is the df is ok
df_chevrolet_final_version.tail()

,Year,Make,Model,Security,Hardware Part Number,Is this hardware clonable?
651,2013,Chevrolet,Malibu,PK3,E39A-H98S,Yes
652,2013,Chevrolet,Sonic,PK3,E78-AB8,Yes
653,2013,Chevrolet,Spark,PK3,E83,Yes
654,2013,Chevrolet,Cruze,PK3,E78-AB8,Yes
655,2013,Chevrolet,Volt,PK3,E78-AB8,Yes


In [36]:
# Create a dataframe only with Make: Pontiac
df_pontiac = df_chevrolet_renamed_column[df_chevrolet_renamed_column['Make'] == 'Pontiac']  

In [37]:
# Check is the df is ok
df_pontiac.tail()

,Year,Make,Model,Security,Hardware Part Number,Is this hardware clonable?
514,2009,Pontiac,Vibe,Toyota Immobilizer,Information not available,No
554,2010,Pontiac,G3,Opt,Information not available,No
555,2010,Pontiac,G6,PK3,Information not available,No
556,2010,Pontiac,G8,PK3,Information not available,No
557,2010,Pontiac,Solstice,PK3,Information not available,No


In [38]:
# Function to create dfs based on a given make
def create_df_make(df, make):
    df_make = df[df['Make'] == make]
    return df_make

In [39]:
# Call the function to create the df based on the given make
df_cadillac =  create_df_make(df_chevrolet_renamed_column, "Cadillac")

### Double check the strings from all columns: Cadillac 

In [40]:
# Print the columns available
df_cadillac.columns

Index(['Year', 'Make', 'Model', 'Security', 'Hardware Part Number',
       'Is this hardware clonable?'],
      dtype='object')

In [41]:
# Column model
data_from_column(df_cadillac, "Model")

{'ATS',
 'Allante',
 'Brougham',
 'CTS',
 'DTS',
 'Deville',
 'Eldorado',
 'Escalade',
 'Escalade ESV',
 'Escalade EXT',
 'SRX',
 'STS',
 'Seville',
 'XLR',
 'XTS'}

In [42]:
# Column Security
data_from_column(df_cadillac, "Security")

{'EZ', 'KA', 'PK1', 'PK2', 'PK3', 'PL2'}

In [43]:
# Column Hardware Part Number
data_from_column(df_cadillac, "Hardware Part Number")

{'E38-AK8',
 'E38-Y2Y',
 'E39-F93S',
 'E39-X94S',
 'E39A-H98S',
 'E55-507',
 'E55-509',
 'E56-752',
 'E67-AH2',
 'E67-TP1',
 'E69-216',
 'E77-420',
 'Information not available',
 'U36-E22',
 'U42C-484',
 'U44G-G03',
 'U44G-G45'}

In [44]:
# Call the function to create the df based on the given make
df_buick =  create_df_make(df_chevrolet_renamed_column, "Buick")

### Double check the strings from all columns: Buick

In [48]:
# Column model
data_from_column(df_buick, "Model")

{'Enclave',
 'Encore',
 'LaCrosse',
 'LeSabre',
 'Lucerne',
 'Park Avenue',
 'Rainier',
 'Reatta',
 'Regal',
 'Rendezvous',
 'Riviera',
 'Roadmaster',
 'Skylark',
 'Terraza',
 'Verano'}

In [49]:
# Column Security
data_from_column(df_buick, "Security")

{'PK1', 'PK2', 'PK3', 'PL1', 'PL2'}

In [51]:
# Column Hardware Part Number
data_from_column(df_buick, "Hardware Part Number")

{'E37',
 'E39-F93S',
 'E39-X94S',
 'E39A-H98S',
 'E55-507',
 'E67-TP1',
 'E69-216',
 'E78-AB8',
 'Information not available',
 'P10',
 'U30C',
 'U32-C55',
 'U32-P05',
 'U32-XC2',
 'U40-051',
 'U40-399',
 'U42C-800',
 'U44C-482',
 'U44C-721',
 'U44G-G45'}

In [52]:
# Call the function to create the df based on the given make
df_oldsmobile =  create_df_make(df_chevrolet_renamed_column, "Oldsmobile")

### Double check the strings from all columns: Oldsmobile

In [54]:
# Column model
data_from_column(df_oldsmobile, "Model")

{'Achieva',
 'Alero',
 'Aurora',
 'Bravada',
 'Cutlass Supreme',
 'Eighty-Eight',
 'Intrigue',
 'Ninety-Eight',
 'Silhouette',
 'Toronado'}

In [55]:
# Column Security
data_from_column(df_oldsmobile, "Security")

{'PK1', 'PK2', 'PK3', 'PL1', 'PL2'}

In [56]:
# Column Hardware Part Number
data_from_column(df_oldsmobile, "Hardware Part Number")

{'Information not available',
 'P10',
 'U30C',
 'U41',
 'U42C-484',
 'U42C-800',
 'U44C-482',
 'U44C-721',
 'U44R'}

In [58]:
# Call the function to create the df based on the given make
df_gmc =  create_df_make(df_chevrolet_renamed_column, "GMC")

### Double check the strings from all columns: GMC

In [59]:
# Column model
data_from_column(df_gmc, "Model")

{'Acadia',
 'Canyon',
 'Envoy',
 'Envoy XUV',
 'Jimmy',
 'Safari',
 'Savana',
 'Sierra',
 'Sonoma',
 'Suburban',
 'Terrain',
 'Yukon Denali',
 'Yukon XL',
 'Yukon/Denali'}

In [60]:
# Column Security
data_from_column(df_gmc, "Security")

{'PK3', 'PL2', 'PL3'}

In [61]:
# Column Hardware Part Number
data_from_column(df_gmc, "Hardware Part Number")

{'E35-LMM',
 'E37',
 'E38-AK8',
 'E38-Y2Y',
 'E39-F93S',
 'E39-X94S',
 'E39A-H98S',
 'E40-679',
 'E40-883',
 'E67-TP1',
 'E69-216',
 'E86-A24',
 'Information not available',
 'P10',
 'P12-N64',
 'P12-N77',
 'U34-588',
 'U41',
 'U44B',
 'U44G-G03',
 'U44G-G45',
 'U44R'}

In [62]:
# Call the function to create the df based on the given make
df_saturn =  create_df_make(df_chevrolet_renamed_column, "Saturn")

### Double check the strings from all columns: Saturn

In [63]:
# Column model
data_from_column(df_saturn, "Model")

{'Astra',
 'Aura',
 'ION',
 'L Series',
 'Outlook',
 'Relay',
 'S Series',
 'Sky',
 'Vue'}

In [64]:
# Column Security
data_from_column(df_saturn, "Security")

{'Opel', 'PK3', 'PL2', 'PL3'}

In [65]:
# Column Hardware Part Number
data_from_column(df_saturn, "Hardware Part Number")

{'E16A-26S',
 'E38-Y2Y',
 'E56-242',
 'E67-TP1',
 'E69-216',
 'E69-326',
 'E69-438',
 'Information not available',
 'P11',
 'U22-143',
 'U22-TN7',
 'U26-698',
 'U29A',
 'U32-XC2',
 'U62-C20'}

In [66]:
# Call the function to create the df based on the given make
df_hummer =  create_df_make(df_chevrolet_renamed_column, "Hummer")

### Double check the strings from all columns: Hummer

In [67]:
# Column model
data_from_column(df_hummer, "Model")

{'H2', 'H2 SUT', 'H3'}

In [68]:
# Column Security
data_from_column(df_hummer, "Security")

{'PK3', 'PL2', 'PL3'}

In [69]:
# Column Hardware Part Number
data_from_column(df_hummer, "Hardware Part Number")

{'E67-TP1',
 'Information not available',
 'P12-N64',
 'P12-N77',
 'U36-E22',
 'U44G-G03',
 'U44G-G45'}

In [ ]:
# Print the list with all Chevrolet list to confirm if all dfs were created
chevrolet_list

['Chevrolet',
 'Pontiac',
 'Cadillac',
 'Buick',
 'Oldsmobile',
 'GMC',
 'Saturn',
 'Hummer']

In [ ]:
# Loop to confirm the missing Chevrolet makes to create the df
for make in chevrolet_list:
    # Condition under the makes already created the df
    if make not in ['Cadillac', 'Buick', 'Oldsmobile', 'GMC', 'Saturn', 'Hummer']:
        # Print the make not created yet
        print(make)

Chevrolet
Pontiac


In [73]:
# Call the function to create the df based on the given make
df_chevrolet =  create_df_make(df_chevrolet_renamed_column, "Chevrolet")

### Double check the strings from all columns: Chevrolet

In [74]:
# Column model
data_from_column(df_chevrolet, "Model")

{'Avalanche',
 'C/K Truck',
 'Camaro',
 'Caprice (Police)',
 'Caprice/SS',
 'Captiva',
 'Cavalier',
 'Classic',
 'Cobalt',
 'Colorado',
 'Corvette',
 'Cruze',
 'Equinox',
 'Express',
 'HHR',
 'Impala',
 'Lumina',
 'Malibu',
 'Monte Carlo',
 'SSR',
 'Sonic',
 'Spark',
 'Suburban',
 'Tahoe',
 'Trailblazer',
 'Traverse',
 'Uplander',
 'Venture',
 'Volt'}

In [75]:
# Column Security
data_from_column(df_chevrolet, "Security")

{'KA', 'PK1', 'PK2', 'PK3', 'PL1', 'PL2', 'PL3', 'VATS'}

In [76]:
# Column Hardware Part Number
data_from_column(df_chevrolet, "Hardware Part Number")

{'E16A-14C',
 'E35-LMM',
 'E37',
 'E38-AK8',
 'E38-Y2Y',
 'E39-F93S',
 'E39-X94S',
 'E39A-H98S',
 'E40-883',
 'E67-TP1',
 'E69-216',
 'E69-326',
 'E77-420',
 'E78-AB8',
 'E83',
 'E86-A24',
 'Information not available',
 'P10',
 'P11',
 'P12-N64',
 'P12-N77',
 'U30B',
 'U30C',
 'U32-XC2',
 'U32-XJ5',
 'U40-051',
 'U40-399',
 'U41',
 'U42C-800',
 'U44C-482',
 'U44C-721',
 'U44G-G03',
 'U44G-G45',
 'U44R'}

In [80]:
# Call the function to create the df based on the given make
df_pontiac =  create_df_make(df_chevrolet_renamed_column, "Pontiac")

### Double check the strings from all columns: Pontiac

In [81]:
# Column model
data_from_column(df_pontiac, "Model")

{'Aztek',
 'Bonneville',
 'Firebird',
 'G3',
 'G5',
 'G6',
 'G8',
 'GTO',
 'Grand Am',
 'Montana',
 'Montana SV6',
 'Solstice',
 'Sunfire',
 'Torrent',
 'Vibe'}

In [82]:
# Column Security
data_from_column(df_pontiac, "Security")

{'GTA only',
 'Holden',
 'Information not available',
 'Opt',
 'PK1',
 'PK2',
 'PK3',
 'PL1',
 'Toyota Immobilizer'}

In [83]:
# Column Hardware Part Number
data_from_column(df_pontiac, "Hardware Part Number")

{'E38-Y2Y',
 'E40-679',
 'E40-883',
 'E67-TP1',
 'E69-326',
 'E77-420',
 'E83',
 'Information not available',
 'U30B',
 'U30C',
 'U32-XC2',
 'U32-XJ5',
 'U42C-800',
 'U44C-482',
 'U44C-721',
 'U44G-G45'}

In [ ]:
# Export the all dfs to .csv file
# df_chevrolet_renamed_column.to_csv('C:\Language_Projects\Language_Projects\Python\Flagship_1\\vehicle_security_system_data_cleaning\data\df_chevrolet.csv')
# df_pontiac.to_csv('C:\Language_Projects\Language_Projects\Python\Flagship_1\\vehicle_security_system_data_cleaning\data\df_pontiac.csv')
# df_cadillac.to_csv('C:\Language_Projects\Language_Projects\Python\Flagship_1\\vehicle_security_system_data_cleaning\data\df_cadillac.csv')
# df_buick.to_csv('C:\Language_Projects\Language_Projects\Python\Flagship_1\\vehicle_security_system_data_cleaning\data\df_buick.csv')
# df_oldsmobile.to_csv('C:\Language_Projects\Language_Projects\Python\Flagship_1\\vehicle_security_system_data_cleaning\data\df_oldsmobile.csv')
# df_gmc.to_csv('C:\Language_Projects\Language_Projects\Python\Flagship_1\\vehicle_security_system_data_cleaning\data\df_gmc.csv')
# df_saturn.to_csv('C:\Language_Projects\Language_Projects\Python\Flagship_1\\vehicle_security_system_data_cleaning\data\df_saturn.csv')
# df_hummer.to_csv('C:\Language_Projects\Language_Projects\Python\Flagship_1\\vehicle_security_system_data_cleaning\data\df_hummer.csv')

<>:2: SyntaxWarning: invalid escape sequence '\L'
<>:3: SyntaxWarning: invalid escape sequence '\L'
<>:4: SyntaxWarning: invalid escape sequence '\L'
<>:5: SyntaxWarning: invalid escape sequence '\L'
<>:6: SyntaxWarning: invalid escape sequence '\L'
<>:7: SyntaxWarning: invalid escape sequence '\L'
<>:8: SyntaxWarning: invalid escape sequence '\L'
<>:9: SyntaxWarning: invalid escape sequence '\L'
<>:2: SyntaxWarning: invalid escape sequence '\L'
<>:3: SyntaxWarning: invalid escape sequence '\L'
<>:4: SyntaxWarning: invalid escape sequence '\L'
<>:5: SyntaxWarning: invalid escape sequence '\L'
<>:6: SyntaxWarning: invalid escape sequence '\L'
<>:7: SyntaxWarning: invalid escape sequence '\L'
<>:8: SyntaxWarning: invalid escape sequence '\L'
<>:9: SyntaxWarning: invalid escape sequence '\L'
C:\Users\Weverson Barbieri\AppData\Local\Temp\ipykernel_16596\1658462114.py:2: SyntaxWarning: invalid escape sequence '\L'
  df_chevrolet_renamed_column.to_csv('C:\Language_Projects\Language_Projects\